# Clasificación binaria

Otro tipo de problema que se suele resolver con redes neuronales es el de clasificación binaria. Ahora no tenemos un valor que predecir, sino que en función de las entradas tenemos que clasificar los datos en dos clases distintas

Importamos la base de datos de tipos de cancer

In [1]:
from sklearn import datasets

cancer = datasets.load_breast_cancer()

Podemos ver qué trae esta base de datos

In [2]:
cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [3]:
print(cancer['target_names'])

['malignant' 'benign']


La llave `DESCR` es una descripción de la base de datos

In [4]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

Además tiene las llaves `data` y `target` donde se encuentran los datos anteriormente descritos. La llave `feature_names` contiene los numbres de cada una de las características

Así que creamos un dataframe con los datos

In [5]:
import pandas as pd

cancer_df = pd.DataFrame(cancer['data'], columns=cancer['feature_names'])
cancer_df['type'] = cancer['target']
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,type
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Vemos las posibles clases que hay

In [6]:
print(cancer.target_names)

['malignant' 'benign']


Vemos cuantos elementos hay de cada clase

In [7]:
cancer_df['type'].value_counts()

type
1    357
0    212
Name: count, dtype: int64

Por último vemos si hay algún dato faltante

In [8]:
cancer_df.isnull().sum()

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
type                       0
dtype: int64

## Dataset y Dataloader

Creamos el dataset

In [9]:
import torch

class CancerDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        cols = [col for col in dataframe.columns if col != 'target']
        self.parameters = torch.from_numpy(dataframe[cols].values).type(torch.float32)
        self.targets = torch.from_numpy(dataframe['type'].values).type(torch.float32)
        self.targets = self.targets.reshape((len(self.targets), 1))

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        parameters = self.parameters[idx]
        target = self.targets[idx]
        return parameters, target

In [10]:
ds = CancerDataset(cancer_df)
len(ds), len(cancer_df)

(569, 569)

Para poder entrenar hemos visto que necesitamos dividir los datos en un conjunto de datos de entrenamiento y en un conjunto de datos de validación. Así que dividimos nuestros datos en estos dos conjuntos.

Como no tenemos muchos datos vamos a dividir el conjunto de datos en un 80% para entrenamiento entrenamiento y un 20% para validación

In [11]:
train_ds, valid_ds = torch.utils.data.random_split(ds, [int(0.8*len(ds)), len(ds) - int(0.8*len(ds))], generator=torch.Generator().manual_seed(42))
len(train_ds), len(valid_ds), len(train_ds) + len(valid_ds)

(455, 114, 569)

Vamos a ver una muestra

In [12]:
sample = train_ds[0]
print(f"len(sample): {len(sample)}")

parameters, target = sample
print(f"parameters: {parameters}\ntype parameters: {type(parameters)}\nparameters.dtype: {parameters.dtype}\nparameters.shape: {parameters.shape}\n\n")
print(f"target: {target}, type target: {type(target)}, target.dtype: {target.dtype}, target.shape: {target.shape}")

len(sample): 2
parameters: tensor([1.1420e+01, 2.0380e+01, 7.7580e+01, 3.8610e+02, 1.4250e-01, 2.8390e-01,
        2.4140e-01, 1.0520e-01, 2.5970e-01, 9.7440e-02, 4.9560e-01, 1.1560e+00,
        3.4450e+00, 2.7230e+01, 9.1100e-03, 7.4580e-02, 5.6610e-02, 1.8670e-02,
        5.9630e-02, 9.2080e-03, 1.4910e+01, 2.6500e+01, 9.8870e+01, 5.6770e+02,
        2.0980e-01, 8.6630e-01, 6.8690e-01, 2.5750e-01, 6.6380e-01, 1.7300e-01,
        0.0000e+00])
type parameters: <class 'torch.Tensor'>
parameters.dtype: torch.float32
parameters.shape: torch.Size([31])


target: tensor([0.]), type target: <class 'torch.Tensor'>, target.dtype: torch.float32, target.shape: torch.Size([1])


Creamos ahora el dataloader

In [13]:
from torch.utils.data import DataLoader

BS_train = 64
BS_val = 128 # Solo hay 114 datos de validación, por lo que no se puede dividir en batches

train_dl = DataLoader(train_ds, batch_size=BS_train, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=BS_val, shuffle=False)

Vemos un batch

In [14]:
batch = next(iter(train_dl))
parameters, target = batch[0], batch[1]
type(parameters), parameters.dtype, parameters.shape, type(target), target.shape

(torch.Tensor,
 torch.float32,
 torch.Size([64, 31]),
 torch.Tensor,
 torch.Size([64, 1]))

## Red Neuronal

Creamos una red neuronal para entrenarla

Ahora en la última capa de la red neuronal ponemos una capa con una función de activación, en concreto la función `sigmoid`. ¿Esto por qué? Como ya hemos visto antes, la salida de la función `sigmoid` tiene valores entre 0 y 1

![sigmoid](Imagenes/sigmoid.png)

Como estamos en un problema de clasificación binaria, a la salida queremos que la red nos de valores entre 0 y 1, por lo que esta función es perfecta para esto. Otra opción es usar una función escalón, con la cual si a la entrada se tiene un valor menos que x a la salida se tendrá un 0, y si es mayor se tendrá un 1. Pero como necesitamos poder derivar todo para poder calcular los gradientes, para el entrenamiento, es mejor usar la función `sigmoid`.

In [15]:
from torch import nn

class CancerNeuralNetwork(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_layers=[100, 50, 20]):
        super().__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, hidden_layers[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[0], hidden_layers[1]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[1], hidden_layers[2]),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layers[2], num_outputs),
        )
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        logits = self.network(x)
        probs = self.activation(logits)
        return logits, probs

La función `sigmoid` da un 0 a la salida si su entrada tiene un valor muy pequeño (o muy negativo, como se entienda mejor) y da un 1 a la salida cuando a la entrada se tiene un valor muy grande. Mientras que si a la entrada se tienen valores cercanos a 0, a la salida no se tendrá un 0 o 1 claro. Por lo que durante el entrenamiento la red intentará ajustar los pesos de las capas anteriores, para que a la última capa, la `sigmoid`, le entren o valores muy pequeños (o muy negativos) o valores muy grandes. Es decir, intentará que los valores de logits sean muy pequeños (o muy negativos) o sean muy grandes

Al igual que antes hemos definido una red genérica a la que hay que meterle los tamaños de entrada, de salida, y opcionalmente los tamaños de la capa oculta. Vamos a ver qué tamaño necesitamos a la entrada y a la salida de la red

Un batch tiene unos parámetros con este tamaño

In [16]:
parameters.shape

torch.Size([64, 31])

Tenemos una matriz de tamaño 64x31. 64 es el tamaño del batch size, mientras que 31 es el número de parámetros, por lo que **a la entrada necesitamos 31 neuronas**

Otra forma de verlo es que como se tiene que hacer una multiplicación matricial de las entradas con la primera capa de la red, si la matriz de entradas tiene un tamaño de 64x31, la matriz que representa las neuronas de la primera capa tiene que tener un tamaño de 31xM. Ya que en una multiplicación matricial, el tamaño de las matrices que se multiplican tienen que ser AxB y BxC, es decir, la dimensión de en medio de las dos matrices tiene que ser la misma

Por otro lado, el mismo batch a la salida tiene un target con este tamaño

In [17]:
target.shape

torch.Size([64, 1])

64 es el tamaño del batch size, pero hay 1 clase, por lo que **a la salida queremos que haya 1 neurona**

In [18]:
num_inputs = parameters.shape[1]
num_outputs = target.shape[1]
model = CancerNeuralNetwork(num_inputs, num_outputs)

model

CancerNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=31, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=1, bias=True)
  )
  (activation): Sigmoid()
)

Primero cogemos un batch del dataloader y se lo metemos a la red a ver si funciona y la hemos definido bien

In [19]:
logits, probs = model(parameters)
logits.shape, probs.shape

(torch.Size([64, 1]), torch.Size([64, 1]))

Si se puede se manda la red a la GPU

In [20]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model.to(device)

Using cuda device


CancerNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=31, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=1, bias=True)
  )
  (activation): Sigmoid()
)

Ahora volvemos a probar a meterle un batch

In [21]:
parameters_gpu = parameters.to(device)
logits, probs = model(parameters_gpu)
logits.shape, probs.shape

(torch.Size([64, 1]), torch.Size([64, 1]))

## Función de pérdida y optimizador

Definimos una función de pérdida y un optimizador

Para este tipo de problemas no debemos usar como función de pérdida el `MSE`, ya que a la salida vamos a tener 1s y 0s. El MSE mide la distancia entre lo predicho por la red y la realidad, pero en este problema la distancia siempre va a ser de 1 en caso de que la predicción sea mala o 0 en caso de que la predicción sea buena.

Como hemos visto en el tema de Funciones de activación, debemos usar `BCE`. Ya que se usa para problemas de clasificación

In [22]:
LR = 1e-3

loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)


## Ciclo de entrenamiento

Entrenamos la red

In [23]:
num_prints = 4

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        _, probs = model(X)
        loss = loss_fn(probs, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % int(len(dataloader)/num_prints) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            _, probs = model(X)
            test_loss += loss_fn(probs, y).item()
            correct += (probs.round() == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Entrenamos

In [24]:
epochs = 13
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
    val_loop(val_dl, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.104998  [    0/  455]
loss: 4.165802  [  128/  455]
loss: 0.672075  [  256/  455]
loss: 0.624762  [  384/  455]
Test Error: 
 Accuracy: 40.4%, Avg loss: 0.780051 

Epoch 2
-------------------------------
loss: 0.830134  [    0/  455]
loss: 0.526276  [  128/  455]
loss: 0.947459  [  256/  455]
loss: 0.829670  [  384/  455]
Test Error: 
 Accuracy: 40.4%, Avg loss: 0.801540 

Epoch 3
-------------------------------
loss: 0.923409  [    0/  455]
loss: 0.807727  [  128/  455]
loss: 0.741083  [  256/  455]
loss: 0.568708  [  384/  455]
Test Error: 
 Accuracy: 79.8%, Avg loss: 0.553853 

Epoch 4
-------------------------------
loss: 0.563830  [    0/  455]
loss: 0.595183  [  128/  455]
loss: 0.524305  [  256/  455]
loss: 0.523095  [  384/  455]
Test Error: 
 Accuracy: 60.5%, Avg loss: 0.782228 

Epoch 5
-------------------------------
loss: 0.793078  [    0/  455]
loss: 0.575853  [  128/  455]
loss: 0.521104  [  256/  455]
loss: 0.541925  [  384

# Función de pérdida

Como hemos visto en el tema de funciones de pérdida, es mejor usar la función `BCEWithLogitsLoss` que `BCELoss`, ya que es más estable numéricamente. Así que ahora, en vez de quedarnos con las probabilidades de la red calculadas mediante la función `Sigmoid`, lo que vamos a hacer es quedarnos con los logits y `BCEWithLogitsLoss` hará el cálculo del la `Sigmoid` y `BCELoss`.

In [25]:
model = CancerNeuralNetwork(num_inputs, num_outputs)
model.to(device)

CancerNeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=31, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=20, bias=True)
    (5): ReLU()
    (6): Linear(in_features=20, out_features=1, bias=True)
  )
  (activation): Sigmoid()
)

In [26]:
LR = 1e-3

loss_fn2 = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

Ahora tenemos que redefinir las funciones de entrenamiento y validación, para quedarnos con los logits de la red y no las probabilidades

In [27]:
num_prints = 4

def train_loop2(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # X and y to device
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        logits, _ = model(X)
        loss = loss_fn2(logits, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % int(len(dataloader)/num_prints) == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val_loop2(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            logits, probs = model(X)
            test_loss += loss_fn2(logits, y).item()
            correct += (probs.round() == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Como se puede ver en el bucle de entrenamiento ya no nos quedamos con las probabilidades, sino con los logits

In [28]:
epochs = 14
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop2(train_dl, model, loss_fn2, optimizer)
    val_loop2(val_dl, model, loss_fn2)
print("Done!")

Epoch 1
-------------------------------
loss: 1.670341  [    0/  455]
loss: 0.966830  [  128/  455]
loss: 0.698714  [  256/  455]
loss: 0.578295  [  384/  455]
Test Error: 
 Accuracy: 40.4%, Avg loss: 0.932089 

Epoch 2
-------------------------------
loss: 1.180747  [    0/  455]
loss: 0.557286  [  128/  455]
loss: 0.547688  [  256/  455]
loss: 0.454823  [  384/  455]
Test Error: 
 Accuracy: 76.3%, Avg loss: 0.554421 

Epoch 3
-------------------------------
loss: 0.556460  [    0/  455]
loss: 0.479203  [  128/  455]
loss: 0.466049  [  256/  455]
loss: 0.568023  [  384/  455]
Test Error: 
 Accuracy: 64.0%, Avg loss: 0.539169 

Epoch 4
-------------------------------
loss: 0.607195  [    0/  455]
loss: 0.474415  [  128/  455]
loss: 0.507968  [  256/  455]
loss: 0.517431  [  384/  455]
Test Error: 
 Accuracy: 65.8%, Avg loss: 0.640143 

Epoch 5
-------------------------------
loss: 0.558246  [    0/  455]
loss: 0.414196  [  128/  455]
loss: 0.533811  [  256/  455]
loss: 0.468989  [  384

Vamos a meterle un dato del dataset de validación a ver qué tal lo hace la red

In [42]:
valid_parameters, valid_target = next(iter(val_dl))

predictions = model(valid_parameters.to(device))
predictions_logits, predictions_probs = predictions[0], predictions[1]

In [43]:
valid_target.shape, predictions_probs.shape

(torch.Size([114, 1]), torch.Size([114, 1]))

In [63]:
print(f"para todo el batch de validación")
for i in range(len(valid_target)):
    if int(valid_target[i].item()) == int(predictions_probs[i].round().item()):
        string = "OK"
    else:
        string = "ERROR"
    print(f"\t se esperaba target: {int(valid_target[i].item())}, y se ha predicho: {int(predictions_probs[i].round().item())} - {string}")

para todo el batch de validación
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 0, y se ha predicho: 0 - OK
	 se esperaba target: 0, y se ha predicho: 0 - OK
	 se esperaba target: 0, y se ha predicho: 1 - ERROR
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 0, y se ha predicho: 1 - ERROR
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 0, y se ha predicho: 1 - ERROR
	 se esperaba target: 1, y se ha predicho: 1 - OK
	 se esperaba target: 0, y se ha predicho: 1 - ERROR
	 se 

Predice bastantes bien